# Algoritmos Genéticos

## Estrutura básica do AG

```
1. t = 0
2. Inicializar a população inicial P_0
3. Enquanto critério de parada == falso
   a. Avaliar a população(Pt)
   b. P' = Selecionar pais(Pt)
   c. F = Aplicar recombinação e mutação(P')
   d. Avaliar a população(F)
   e. Pt+1 = Selecionar sobreviventes(Pt + F)
   f. t = t + 1
```

In [1]:
# Importa a biblioteca para criação de classes de dados
from dataclasses import dataclass

# Importação dos tipos de dados para o type hint
from typing import Callable

# Importação da biblioteca de números aleatórios
from random import randint
from random import sample
from random import random

# Importação da biblioteca de impressão bonita
from pprint import pprint

# Importação da biblioteca de cópia
from copy import deepcopy

In [2]:
for i in range(10):
    print(randint(0, 1))

0
0
0
1
0
0
1
0
1
0


In [3]:
sample([1, 2, 3, 4, 5, 6], 3)

[2, 3, 5]

In [4]:
@dataclass 
class Cromossomo:
    dados: list[int]
    fitness: float = 0

In [5]:
@dataclass
class Config:
    tam_cromossomo: int
    tam_populacao: int

    max_iteracoes: int
    max_congelamento : int
    
    fitness: Callable[[Cromossomo], float]

    selecionar_pais: Callable[[list[Cromossomo]], list[tuple[Cromossomo, Cromossomo]]]
    
    aplicar_cruzamento: Callable[[list[tuple[Cromossomo, Cromossomo]]], list[Cromossomo]]
    aplicar_mutacao: Callable[[list[Cromossomo], float], None]
    taxa_mutacao: float
    
    selecionar_sobreviventes: Callable[[list[Cromossomo]], list[Cromossomo]]
    

## Inicialização da população

In [6]:
def inicializar_populacao(tam_populacao, tam_cromossomo) -> list[Cromossomo]:
    P0 = []
    for i in range(tam_populacao):
        cromossomo = Cromossomo([randint(0, 1) for j in range(tam_cromossomo)])
        P0.append(cromossomo)
    return P0

In [7]:
print(inicializar_populacao(4, 8))

[Cromossomo(dados=[1, 0, 1, 0, 1, 0, 0, 1], fitness=0), Cromossomo(dados=[0, 1, 1, 1, 0, 0, 0, 0], fitness=0), Cromossomo(dados=[1, 0, 1, 0, 1, 1, 1, 0], fitness=0), Cromossomo(dados=[1, 0, 1, 0, 0, 1, 0, 0], fitness=0)]


## Seleção dos pais

In [8]:
# Seleção por torneio - com reposição
# - Seleciona k pais e o tiver melhor fitness é selecionado para formar casal
def torneio(populacao: list[Cromossomo], tam_torneio: int = 3) -> list[tuple[Cromossomo, Cromossomo]]:
    casais = []

    for i in range(len(populacao) // 2):

        torneio1 = sample(populacao, tam_torneio * 2)
        pai1 = sorted(torneio1, key=lambda x: x.fitness, reverse=True)[0] 
        
        torneio2 = sample(populacao, tam_torneio)
        pai2 = sorted(torneio2, key=lambda x: x.fitness, reverse=True)[0] 

        # TODO: evitar que o selecionado no pai1 seja selecionado como pai2

        casais.append((pai1, pai2))
    
    return casais

In [9]:
## Crossover

In [10]:
def crossover_1_corte(casais):
    filhos = []

    for par in casais:

        # Desempacota os pais
        pai1 = par[0].dados
        pai2 = par[1].dados

        # Sorteia o ponto de corte
        corte = randint(1, len(pai1) - 1)

        filho1 = pai1[:corte] + pai2[corte:] 
        filho2 = pai2[:corte] + pai1[corte:] 

        filhos.append(Cromossomo(filho1))
        filhos.append(Cromossomo(filho2))

    return filhos

In [11]:
## Mutação

In [12]:
def mutacao(cromossomos: list[Cromossomo], taxa_mutacao: float):
    for cromossomo in cromossomos:
        for i, c in enumerate(cromossomo.dados):
            if random() < taxa_mutacao:
                if c == 1:
                    cromossomo.dados[i] = 0
                else:
                    cromossomo.dados[i] = 1
                        
    

In [13]:
## Seleção dos sobreviventes

In [14]:
def elitismo(populacao):
    populacao.sort(key=lambda x: x.fitness, reverse=True)
    return populacao[:len(populacao)//2]

In [16]:
def algoritmo_genetico(config: Config) -> list[int]:
    """
    Implementação do algoritmo genético clássico.

    Arguments:
        config: Parâmetros de configuração do AG.
    """

    # 1. t = 0
    t = 0
    
    # 2. Inicializar a população inicial P_0
    P = inicializar_populacao(config.tam_populacao, config.tam_cromossomo)

    # a. Avaliar a população(Pt)
    for c in P:
        c.fitness = config.fitness(c)

    # Salva o melhor indivíduo 
    P.sort(key=lambda x: x.fitness, reverse=True)

    melhor_individuo = deepcopy(P[0])
    cont_congelamento = 0
    
    
    # 3. Enquanto critério de parada == falso
    terminou = False
    while (t < config.max_iteracoes 
               and cont_congelamento < config.max_congelamento):
        
        # b. P' = Selecionar pais(Pt)
        casais = config.selecionar_pais(P)
            
        # c. F = Aplicar recombinação e mutação(P')
        F = config.aplicar_cruzamento(casais)
        config.aplicar_mutacao(F, config.taxa_mutacao)
        
        # d. Avaliar a população(F)
        for c in F:
            c.fitness = config.fitness(c)
        
        # e. Pt+1 = Selecionar sobreviventes(Pt + F)
        P = config.selecionar_sobreviventes(P + F)

        # Salva o melhor indivíduo 
        P.sort(key=lambda x: x.fitness, reverse=True)

        if P[0].fitness > melhor_individuo.fitness:
            melhor_individuo = deepcopy(P[0])
            cont_congelamento = 0
        else:
            cont_congelamento += 1

        print(f'{t:04d} - {melhor_individuo.fitness} - {melhor_individuo.dados}')
        
        # f. t = t + 1
        t = t + 1
        
    return melhor_individuo

## Teste do AG

In [25]:
# Problema de maximizar a quantidade de 1s no vetor

def maximizar_1s(cromossomo: Cromossomo) -> int:
    """
    Função de avaliação do problema. É nessa função que o cromossomo é interpretado
    para calcular a aptidão e verificar se está atingindo o objetivo.
    """
    return sum(cromossomo.dados)


# Parâmetros de configuração do AG
config = Config(
    tam_cromossomo=64,
    tam_populacao=100,

    max_iteracoes=1000,
    max_congelamento=100,
    
    fitness=maximizar_1s,

    taxa_mutacao=0.05,
    
    selecionar_pais=torneio,

    aplicar_cruzamento=crossover_1_corte,
    aplicar_mutacao=mutacao,
    selecionar_sobreviventes=elitismo,
)

# Execução do algoritmo genético
solucao = algoritmo_genetico(config)

0000 - 46 - [1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0]
0001 - 48 - [1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1]
0002 - 48 - [1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1]
0003 - 50 - [1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]
0004 - 52 - [1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,

In [27]:
# Parâmetros de configuração do AG
config = Config(
    tam_cromossomo=64,
    tam_populacao=8,

    max_iteracoes=100,
    max_congelamento=10,
    
    fitness=maximizar_1s,

    taxa_mutacao=0.05,
    
    selecionar_pais=torneio,

    aplicar_cruzamento=crossover_1_corte,
    aplicar_mutacao=mutacao,
    selecionar_sobreviventes=elitismo,
)

# Execução do algoritmo genético
solucao = algoritmo_genetico(config)

0000 - 40 - [0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1]
0001 - 43 - [1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1]
0002 - 47 - [0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1]
0003 - 48 - [0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1]
0004 - 49 - [1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1,

Um paladino lv 12 está roubando uma loja e encontra 10 itens valiosos. Cada item possui os seguintes valores e pesos:

| Item                  | Valor | Peso |
|-----------------------|-------|------|
| Anel flamejante       | $55   | 95kg |
| Escudo Leve           | $10   | 4kg  |
| Escudo Pesado         | $47   | 60kg |
| Espada Bastarda       | $5    | 32kg |
| Espada Curta          | $4    | 23kg |
| Chapéu da Modificação | $50   | 72kg |
| Talismã de Pinheiro   | $8    | 80kg |
| Moedas Pacifistas     | $61   | 62kg |
| Anel Guerreiro        | $85   | 65kg |
| Anel da Regeneração   | $87   | 46kg |

Ele quer levar uma carga tão valiosa quanto possível, mas ele pode carregar no máximo 269kg em sua mochila sem que ela rasgue. Quais itens devem ser levados? [ref]

In [30]:
# Problema da mochila

# Fenótipo: cada posição representa um item da lista e o valor representa se o 
#           item está ou não na mochila
#           Por exemplo, a posição 0 representa o Anel flamejante

itens = [
    {'item': 'Anel flamejante'      , 'preco': 55  , 'peso': 95 },
    {'item': 'Escudo Leve'          , 'preco': 10  , 'peso': 4  },
    {'item': 'Escudo Pesado'        , 'preco': 47  , 'peso': 60 },
    {'item': 'Espada Bastarda'      , 'preco': 5   , 'peso': 32 },
    {'item': 'Espada Curta'         , 'preco': 4   , 'peso': 23 },
    {'item': 'Chapéu da Modificação', 'preco': 50  , 'peso': 72 },
    {'item': 'Talismã de Pinheiro'  , 'preco': 8   , 'peso': 80 },
    {'item': 'Moedas Pacifistas'    , 'preco': 61  , 'peso': 62 },
    {'item': 'Anel Guerreiro'       , 'preco': 85  , 'peso': 65 },
    {'item': 'Anel da Regeneração'  , 'preco': 87  , 'peso': 46 },
]

# Genótipo: Cromossomo binário de 10 posições

CAPACIDADE_MOCHILA = 269

def fitness_mochila(cromossomo: Cromossomo) -> float:
    """
    Avalia o valor total dos itens da mochila/cromossomo/individuo. 
    Se o indivíduo for inválido, retorna -1.
    """
    valor_total = 0
    peso_total = 0

    # Percorre todas as posições do cromossomo verificando os itens que 
    # estão na mochila. Se o item estiver na mochila, registra o peso e o valor
    for i, contem in enumerate(cromossomo.dados):
        
        if contem:
            item = itens[i]
            valor_total += item['preco']
            peso_total += item['peso']

    # Verifica se ultrapassa o peso da mochila
    if peso_total >= CAPACIDADE_MOCHILA:
        return -1
    
    return valor_total




In [41]:
# Parâmetros de configuração do AG
config = Config(
    tam_cromossomo=10,
    tam_populacao=100,

    max_iteracoes=1000,
    max_congelamento=100,
    
    fitness=fitness_mochila,

    taxa_mutacao=0.05,
    
    selecionar_pais=torneio,

    aplicar_cruzamento=crossover_1_corte,
    aplicar_mutacao=mutacao,
    selecionar_sobreviventes=elitismo,
)

# Execução do algoritmo genético
solucao = algoritmo_genetico(config)

# Decodifica a solução
peso = 0
for i, contem in enumerate(solucao.dados):
    if contem:
        print(itens[i])
        peso += itens[i]['peso']

print(f'Valor total: {solucao.fitness} - {peso}kg / {CAPACIDADE_MOCHILA}kg')

0000 - 293 - [0, 1, 0, 0, 0, 1, 0, 1, 1, 1]
0001 - 294 - [0, 1, 1, 0, 1, 0, 0, 1, 1, 1]
0002 - 294 - [0, 1, 1, 0, 1, 0, 0, 1, 1, 1]
0003 - 294 - [0, 1, 1, 0, 1, 0, 0, 1, 1, 1]
0004 - 294 - [0, 1, 1, 0, 1, 0, 0, 1, 1, 1]
0005 - 294 - [0, 1, 1, 0, 1, 0, 0, 1, 1, 1]
0006 - 294 - [0, 1, 1, 0, 1, 0, 0, 1, 1, 1]
0007 - 294 - [0, 1, 1, 0, 1, 0, 0, 1, 1, 1]
0008 - 294 - [0, 1, 1, 0, 1, 0, 0, 1, 1, 1]
0009 - 294 - [0, 1, 1, 0, 1, 0, 0, 1, 1, 1]
0010 - 294 - [0, 1, 1, 0, 1, 0, 0, 1, 1, 1]
0011 - 294 - [0, 1, 1, 0, 1, 0, 0, 1, 1, 1]
0012 - 294 - [0, 1, 1, 0, 1, 0, 0, 1, 1, 1]
0013 - 294 - [0, 1, 1, 0, 1, 0, 0, 1, 1, 1]
0014 - 294 - [0, 1, 1, 0, 1, 0, 0, 1, 1, 1]
0015 - 294 - [0, 1, 1, 0, 1, 0, 0, 1, 1, 1]
0016 - 294 - [0, 1, 1, 0, 1, 0, 0, 1, 1, 1]
0017 - 294 - [0, 1, 1, 0, 1, 0, 0, 1, 1, 1]
0018 - 294 - [0, 1, 1, 0, 1, 0, 0, 1, 1, 1]
0019 - 294 - [0, 1, 1, 0, 1, 0, 0, 1, 1, 1]
0020 - 294 - [0, 1, 1, 0, 1, 0, 0, 1, 1, 1]
0021 - 294 - [0, 1, 1, 0, 1, 0, 0, 1, 1, 1]
0022 - 294 - [0, 1, 1, 0, 1, 0, 